In [1]:
# if (!requireNamespace("BiocManager", quietly=TRUE)) 
#     install.packages("BiocManager") 

# BiocManager::install("ANCOMBC")

In [1]:
.libPaths(R.home('/opt/homebrew/Caskroom/miniforge/base/lib/R/library'))
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1013. Microbiome Peanut Allergy/P1013.3. Analyses/P1013.3.1. Data Processing/Output'
cur_date = '083025'

library(readxl)
library(openxlsx)
library(tidyverse)
library(factoextra)
library(ANCOMBC) 

# reading in files
saliva_df = data.frame(read_excel("Input/Saliva_Diversity_082925.xlsx", sheet = 2)) %>%
    mutate(Age = as.numeric(Age), IgE = as.numeric(IgE))
stool_df = data.frame(read_excel("Input/Stool_Diversity_082925.xlsx", sheet = 2)) %>%
    mutate(Age = as.numeric(Age), IgE = as.numeric(IgE))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa

Warning message:
“There were 2 warnings in `mutate()`.
The first warning was:
ℹ In argument: `Age = as.numeric(Age)`.
Caused by warning:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 1 remaining warning.”
Warning message:
“There were 2 warnings in `mutate()`.
The first warning was:
ℹ In argument: `Age = as.numeric(Age)`.
Caused by warning:
! NAs introduce

In [2]:
head(saliva_df)

,Original_Sample_ID,Subject_No,Sample_ID,Time_Point,Treatment,Outcome,Age,IgE,IgE_Binarized,k__Bacteria.p__Firmicutes.c__Clostridia.o__Eubacteriales.f__Peptococcaceae.g__Peptococcus.s__sp._HMT_167,⋯,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Pseudoleptotrichia.s__sp._HMT_219,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Pseudoleptotrichia.__,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.s__wadei,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.s__shahii,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.s__sp._HMT_498,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.s__sp._HMT_215,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.s__sp._HMT_392,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.s__sp._HMT_212,k__Bacteria.p__Gracilibacteria_.GN02..c__Gracilibacteria_.GN02._.C.1..o__Gracilibacteria_.GN02._.O.1..f__Gracilibacteria_.GN02._.F.1..g__Gracilibacteria_.GN02._.G.1..s__bacterium_HMT_872,k__Bacteria.p__Firmicutes.c__Bacilli.o__Lactobacillales.f__Lactobacillaceae.g__Lacticaseibacillus.__
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,FS01baseline,1,Pre_1,Pre,P,F,3.838356,41.00,High,0,⋯,0,0,0,0,0,18,15,42,0,0
2,FS0236mo,2,Post_2,Post,A,DS,1.873973,14.20,Low,59,⋯,0,0,0,0,0,532,30,27,0,0
3,FS02baseline,2,Pre_2,Pre,A,DS,1.873973,14.20,Low,0,⋯,0,198,0,0,0,172,0,64,0,0
4,FS0436mo,3,Post_3,Post,P,F,3.715068,67.40,High,0,⋯,0,0,0,0,0,0,0,0,15,0
5,FS04baseline,3,Pre_3,Pre,P,F,3.715068,67.40,High,0,⋯,0,0,0,0,0,0,0,8,0,0
6,FS0536mo,4,Post_4,Post,A,DS,3.991781,9.59,Low,24,⋯,0,0,0,0,0,676,0,207,0,0


Running individual statistics on saliva and stool microbiome samples at baseline to see what species differ across outcomes and if age plays a role. I used the [ANCOM-BC2](https://www.bioconductor.org/packages/release/bioc/vignettes/ANCOMBC/inst/doc/ANCOMBC2.html) package to perform a differential abundance analysis ([ref](https://www.nature.com/articles/s41592-023-02092-7#Sec1)). 

In [3]:
# making the row names the sample ids
baseline_saliva_df = saliva_df %>%
    # only keep participants' baseline samples and those that didn't fail the IgE protein
    filter(Time_Point == 'Pre' & IgE_Binarized != 'Fail') %>%
    # ended up reassinging 'DS' to 'F' due to small sample size
    mutate(Outcome = ifelse(Outcome == 'DS', 'F', Outcome)) %>%
    # making the row names the sample ids
    column_to_rownames(var = 'Sample_ID')

baseline_stool_df = stool_df %>%
    filter(Time_Point == 'Pre' & IgE_Binarized != 'Fail') %>%
    mutate(Outcome = ifelse(Outcome == 'DS', 'F', Outcome)) %>%
    column_to_rownames(var = 'Sample_ID')

baseline_saliva_df$Outcome = factor(baseline_saliva_df$Outcome, levels = c('F', 'SU'))
baseline_stool_df$Outcome = factor(baseline_stool_df$Outcome, levels = c('F', 'SU'))

head(baseline_saliva_df)

,Original_Sample_ID,Subject_No,Time_Point,Treatment,Outcome,Age,IgE,IgE_Binarized,k__Bacteria.p__Firmicutes.c__Clostridia.o__Eubacteriales.f__Peptococcaceae.g__Peptococcus.s__sp._HMT_167,k__Bacteria.p__Actinobacteria.c__Actinomycetia.o__Propionibacteriales.f__Propionibacteriaceae.g__Arachnia.__,⋯,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Pseudoleptotrichia.s__sp._HMT_219,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Pseudoleptotrichia.__,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.s__wadei,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.s__shahii,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.s__sp._HMT_498,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.s__sp._HMT_215,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.s__sp._HMT_392,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.s__sp._HMT_212,k__Bacteria.p__Gracilibacteria_.GN02..c__Gracilibacteria_.GN02._.C.1..o__Gracilibacteria_.GN02._.O.1..f__Gracilibacteria_.GN02._.F.1..g__Gracilibacteria_.GN02._.G.1..s__bacterium_HMT_872,k__Bacteria.p__Firmicutes.c__Bacilli.o__Lactobacillales.f__Lactobacillaceae.g__Lacticaseibacillus.__
,<chr>,<dbl>,<chr>,<chr>,<fct>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Pre_1,FS01baseline,1,Pre,P,F,3.838356,41.00,High,0,10,⋯,0,0,0,0,0,18,15,42,0,0
Pre_2,FS02baseline,2,Pre,A,F,1.873973,14.20,Low,0,19,⋯,0,198,0,0,0,172,0,64,0,0
Pre_3,FS04baseline,3,Pre,P,F,3.715068,67.40,High,0,0,⋯,0,0,0,0,0,0,0,8,0,0
Pre_4,FS05baseline,4,Pre,A,F,3.991781,9.59,Low,51,23,⋯,0,0,0,0,0,78,0,83,0,0
Pre_5,FS06baseline,5,Pre,P,F,2.356164,106.00,High,0,204,⋯,0,0,0,0,0,23,33,286,0,0
Pre_6,FS07baseline,6,Pre,A,SU,1.419178,1.32,Low,0,0,⋯,0,1798,0,0,0,192,0,0,0,0


In [4]:
# setting seed for reproducibility
get_ancombc2_results = function(df, formula, value){
    # add words!!!
    
    set.seed(123)
    # automatically performs a psuedo log transformation
    value2 = value + 1
    output = ancombc2(data = t(df[,value2:ncol(df)]), 
                      meta_data = df[,1:value],
                      tax_level = 'Species', #taxonomic level of interest
                      fix_formula = formula, # specifying the model
                      group = 'Outcome', # independent variable
                      p_adj_method = "holm") # Holm-Bonferroni adj p values

    # removing the standard error and test statistics from the results
    removed_cols = c('se_', 'W_')
    final_df = output$res %>%
        select(!matches(removed_cols))

    return(final_df)
    }

# calling fn
saliva_raw_model = get_ancombc2_results(baseline_saliva_df, 'Outcome',8)
saliva_2cov_model = get_ancombc2_results(baseline_saliva_df, 'Age + IgE + Outcome',8)
stool_raw_model = get_ancombc2_results(baseline_stool_df, 'Outcome',9)
stool_2cov_model = get_ancombc2_results(baseline_stool_df, 'Age + IgE + Outcome',9)

Checking the input data type ...

The input data is of type: matrix

The imported data is in a generic 'matrix'/'data.frame' format.

PASS

Checking the sample metadata ...

The specified variables in the formula: Outcome

The available variables in the sample metadata: Original_Sample_ID, Subject_No, Time_Point, Treatment, Outcome, Age, IgE, IgE_Binarized

PASS

Checking other arguments ...

The number of groups of interest is: 2

Warning message:
“The group variable has < 3 categories 
The multi-group comparisons (global/pairwise/dunnet/trend) will be deactivated”
The sample size per group is: F = 27, SU = 10

PASS

Obtaining initial estimates ...

Estimating sample-specific biases ...

Loading required package: foreach


Attaching package: ‘foreach’


The following objects are masked from ‘package:purrr’:

    accumulate, when


Loading required package: rngtools

Warning message in sqrt(sweep(var_hat, 2, var_delta, "*")):
“NaNs produced”
ANCOM-BC2 primary results ...

Conducting se

## Saliva Raw Results

## Saliva Adjusted Results

In [7]:
# passed sensitivity filter and is significant 
saliva_raw_model %>%
    filter(diff_robust_OutcomeSU == TRUE)

# passed sensitivity analysis with a relaxed filter
saliva_raw_model %>%
    filter(q_OutcomeSU < 0.1 & passed_ss_OutcomeSU == TRUE)

# significant with a relaxed filter
saliva_raw_model %>%
    filter(q_OutcomeSU < 0.1)

taxon,lfc_(Intercept),lfc_OutcomeSU,p_(Intercept),p_OutcomeSU,q_(Intercept),q_OutcomeSU,diff_(Intercept),diff_OutcomeSU,passed_ss_(Intercept),passed_ss_OutcomeSU,diff_robust_(Intercept),diff_robust_OutcomeSU
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>


taxon,lfc_(Intercept),lfc_OutcomeSU,p_(Intercept),p_OutcomeSU,q_(Intercept),q_OutcomeSU,diff_(Intercept),diff_OutcomeSU,passed_ss_(Intercept),passed_ss_OutcomeSU,diff_robust_(Intercept),diff_robust_OutcomeSU
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Porphyromonadaceae.g__Porphyromonas.__,-0.5326396,1.611588,0.01091397,0.000414453,1,0.0609246,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE


taxon,lfc_(Intercept),lfc_OutcomeSU,p_(Intercept),p_OutcomeSU,q_(Intercept),q_OutcomeSU,diff_(Intercept),diff_OutcomeSU,passed_ss_(Intercept),passed_ss_OutcomeSU,diff_robust_(Intercept),diff_robust_OutcomeSU
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Prevotellaceae.g__Alloprevotella.s__sp._HMT_308,0.1434357,-2.310190,0.390295368,0.0001395744,1.0000000,0.02079658,FALSE,TRUE,TRUE,FALSE,FALSE,FALSE
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Porphyromonadaceae.g__Porphyromonas.__,-0.5326396,1.611588,0.010913973,0.0004144530,1.0000000,0.06092460,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE
k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Pseudoleptotrichia.s__sp._HMT_221,-0.7327041,2.342115,0.001605976,0.0002212867,0.2264427,0.03275043,FALSE,TRUE,TRUE,FALSE,FALSE,FALSE


In [8]:
# passed sensitivity filter and is significant 
saliva_2cov_model %>%
    filter(diff_robust_OutcomeSU == TRUE)

# passed sensitivity analysis with a relaxed filter
saliva_2cov_model %>%
    filter(q_OutcomeSU < 0.1 & passed_ss_OutcomeSU == TRUE)

# significant with a relaxed filter
saliva_2cov_model %>%
    filter(q_OutcomeSU < 0.1)

Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


taxon,lfc_(Intercept),lfc_Age,lfc_IgE,lfc_OutcomeSU,p_(Intercept),p_Age,p_IgE,p_OutcomeSU,q_(Intercept),⋯,diff_IgE,diff_OutcomeSU,passed_ss_(Intercept),passed_ss_Age,passed_ss_IgE,passed_ss_OutcomeSU,diff_robust_(Intercept),diff_robust_Age,diff_robust_IgE,diff_robust_OutcomeSU
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>


Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


taxon,lfc_(Intercept),lfc_Age,lfc_IgE,lfc_OutcomeSU,p_(Intercept),p_Age,p_IgE,p_OutcomeSU,q_(Intercept),⋯,diff_IgE,diff_OutcomeSU,passed_ss_(Intercept),passed_ss_Age,passed_ss_IgE,passed_ss_OutcomeSU,diff_robust_(Intercept),diff_robust_Age,diff_robust_IgE,diff_robust_OutcomeSU
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>


Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


taxon,lfc_(Intercept),lfc_Age,lfc_IgE,lfc_OutcomeSU,p_(Intercept),p_Age,p_IgE,p_OutcomeSU,q_(Intercept),⋯,diff_IgE,diff_OutcomeSU,passed_ss_(Intercept),passed_ss_Age,passed_ss_IgE,passed_ss_OutcomeSU,diff_robust_(Intercept),diff_robust_Age,diff_robust_IgE,diff_robust_OutcomeSU
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>


## Stool Raw Results

In [9]:
# passed sensitivity filter and is significant 
stool_raw_model %>%
    filter(diff_robust_OutcomeSU == TRUE)

# passed sensitivity analysis with a relaxed filter
stool_raw_model %>%
    filter(q_OutcomeSU < 0.1 & passed_ss_OutcomeSU == TRUE)

# significant with a relaxed filter
stool_raw_model %>%
    filter(q_OutcomeSU < 0.1)

taxon,lfc_(Intercept),lfc_OutcomeSU,p_(Intercept),p_OutcomeSU,q_(Intercept),q_OutcomeSU,diff_(Intercept),diff_OutcomeSU,passed_ss_(Intercept),passed_ss_OutcomeSU,diff_robust_(Intercept),diff_robust_OutcomeSU
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>


taxon,lfc_(Intercept),lfc_OutcomeSU,p_(Intercept),p_OutcomeSU,q_(Intercept),q_OutcomeSU,diff_(Intercept),diff_OutcomeSU,passed_ss_(Intercept),passed_ss_OutcomeSU,diff_robust_(Intercept),diff_robust_OutcomeSU
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
d__Bacteria.p__Actinobacteriota.c__Actinomycetia.o__Actinomycetales.f__Bifidobacteriaceae.g__Bifidobacterium_388775.s__Bifidobacterium.bifidum,0.8605398,-2.814578,0.000276249,0.0003467552,0.06574727,0.08322125,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE


taxon,lfc_(Intercept),lfc_OutcomeSU,p_(Intercept),p_OutcomeSU,q_(Intercept),q_OutcomeSU,diff_(Intercept),diff_OutcomeSU,passed_ss_(Intercept),passed_ss_OutcomeSU,diff_robust_(Intercept),diff_robust_OutcomeSU
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Oscillospirales.f__Ruminococcaceae.g__Gemmiger_A_73129.s__Gemmiger_A_73129.qucibialis,-1.2676072,4.376294,0.003226683,1.180502e-04,0.74859043,0.02845009,FALSE,TRUE,TRUE,FALSE,FALSE,FALSE
d__Bacteria.p__Actinobacteriota.c__Actinomycetia.o__Actinomycetales.f__Bifidobacteriaceae.g__Bifidobacterium_388775.s__Bifidobacterium.bifidum,0.8605398,-2.814578,0.000276249,3.467552e-04,0.06574727,0.08322125,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Peptostreptococcales.f__Peptostreptococcaceae_256921.g__Terrisporobacter.__,-0.7070190,3.118313,0.061454756,8.179198e-05,1.00000000,0.01979366,FALSE,TRUE,TRUE,FALSE,FALSE,FALSE


## Stool Adjusted Results

In [80]:
# passed sensitivity filter and is significant 
stool_2cov_model %>%
    filter(diff_robust_OutcomeSU == TRUE)

# passed sensitivity analysis with a relaxed filter
stool_2cov_model %>%
    filter(q_OutcomeSU < 0.1 & passed_ss_OutcomeSU == TRUE)

# significant with a relaxed filter
stool_2cov_model %>%
    filter(q_OutcomeSU < 0.1)

Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


taxon,lfc_(Intercept),lfc_Age,lfc_IgE,lfc_OutcomeSU,p_(Intercept),p_Age,p_IgE,p_OutcomeSU,q_(Intercept),⋯,diff_IgE,diff_OutcomeSU,passed_ss_(Intercept),passed_ss_Age,passed_ss_IgE,passed_ss_OutcomeSU,diff_robust_(Intercept),diff_robust_Age,diff_robust_IgE,diff_robust_OutcomeSU
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>


Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


taxon,lfc_(Intercept),lfc_Age,lfc_IgE,lfc_OutcomeSU,p_(Intercept),p_Age,p_IgE,p_OutcomeSU,q_(Intercept),⋯,diff_IgE,diff_OutcomeSU,passed_ss_(Intercept),passed_ss_Age,passed_ss_IgE,passed_ss_OutcomeSU,diff_robust_(Intercept),diff_robust_Age,diff_robust_IgE,diff_robust_OutcomeSU
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>


Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


taxon,lfc_(Intercept),lfc_Age,lfc_IgE,lfc_OutcomeSU,p_(Intercept),p_Age,p_IgE,p_OutcomeSU,q_(Intercept),⋯,diff_IgE,diff_OutcomeSU,passed_ss_(Intercept),passed_ss_Age,passed_ss_IgE,passed_ss_OutcomeSU,diff_robust_(Intercept),diff_robust_Age,diff_robust_IgE,diff_robust_OutcomeSU
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>


## Visualizing Differences in Abundance

In [9]:
?ancombc2

ancombc2 {ANCOMBC},R Documentation
data,"the input data. The data parameter should be either a matrix, data.frame, phyloseq or a TreeSummarizedExperiment object. Both phyloseq and TreeSummarizedExperiment objects consist of a feature table (microbial count table), a sample metadata table, a taxonomy table (optional), and a phylogenetic tree (optional). If a matrix or data.frame is provided, ensure that the row names of the metadata match the sample names (column names if taxa_are_rows is TRUE, and row names otherwise) in data. if a phyloseq or a TreeSummarizedExperiment is used, this standard has already been enforced. For detailed information, refer to ?phyloseq::phyloseq or ?TreeSummarizedExperiment::TreeSummarizedExperiment. It is recommended to use low taxonomic levels, such as OTU or species level, as the estimation of sampling fractions requires a large number of taxa."
taxa_are_rows,logical. Whether taxa are positioned in the rows of the feature table. Default is TRUE.
assay.type,alias for assay_name.
assay_name,"character. Name of the count table in the data object (only applicable if data object is a (Tree)SummarizedExperiment). Default is ""counts"". See ?SummarizedExperiment::assay for more details."
rank,alias for tax_level.
tax_level,"character. The taxonomic or non taxonomic(rowData) level of interest. The input data can be analyzed at any taxonomic or rowData level without prior agglomeration. Note that tax_level must be a value from taxonomyRanks or rowData, which includes ""Kingdom"", ""Phylum"" ""Class"", ""Order"", ""Family"" ""Genus"" ""Species"" etc. See ?mia::taxonomyRanks for more details. Default is NULL, i.e., do not perform agglomeration, and the ANCOM-BC2 analysis will be performed at the lowest taxonomic level of the input data."
aggregate_data,"The abundance data that has been aggregated to the desired taxonomic level. This parameter is required only when the input data is in matrix or data.frame format. For phyloseq or TreeSummarizedExperiment data, aggregation is performed by specifying the tax_level parameter."
meta_data,"a data.frame containing sample metadata. This parameter is mandatory when the input data is a generic matrix or data.frame. Ensure that the row names of the metadata match the sample names (column names if taxa_are_rows is TRUE, and row names otherwise) in data."
fix_formula,"the character string expresses how the microbial absolute abundances for each taxon depend on the fixed effects in metadata. When specifying the fix_formula, make sure to include the group variable in the formula if it is not NULL."
rand_formula,the character string expresses how the microbial absolute abundances for each taxon depend on the random effects in metadata. ANCOM-BC2 follows the lmerTest package in formulating the random effects. See ?lmerTest::lmer for more details. Default is NULL.


In [10]:
# # creating 1 df

# baseline_saliva_df$Sample = 'Saliva'
# baseline_stool_df$Sample = 'Stool'

# combined_df = full_join(baseline_saliva_df, baseline_stool_df)

In [16]:
test_bacterium = saliva_raw_model %>%
    filter(q_OutcomeSU < 0.1)

#ggplot(
baseline_saliva_df %>%
           select(c(colnames(baseline_saliva_df)[1:8],test_bacterium$taxon))

,Original_Sample_ID,Subject_No,Time_Point,Treatment,Outcome,Age,IgE,IgE_Binarized,k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Prevotellaceae.g__Alloprevotella.s__sp._HMT_308,k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Porphyromonadaceae.g__Porphyromonas.__,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Pseudoleptotrichia.s__sp._HMT_221
,<chr>,<dbl>,<chr>,<chr>,<fct>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
Pre_1,FS01baseline,1,Pre,P,F,3.838356,41.00,High,5,0,23
Pre_2,FS02baseline,2,Pre,A,F,1.873973,14.20,Low,78,0,0
Pre_3,FS04baseline,3,Pre,P,F,3.715068,67.40,High,0,0,0
Pre_4,FS05baseline,4,Pre,A,F,3.991781,9.59,Low,0,13,9
Pre_5,FS06baseline,5,Pre,P,F,2.356164,106.00,High,0,32,0
Pre_6,FS07baseline,6,Pre,A,SU,1.419178,1.32,Low,0,387,0
Pre_8,FS09baseline,8,Pre,P,F,1.112329,19.10,High,0,0,0
Pre_9,FS10baseline,9,Pre,A,SU,1.389041,0.42,Low,0,703,0
Pre_10,FS11baseline,10,Pre,P,F,2.191781,1.74,Low,0,0,9
